# Setup

In [2]:
!pip install -q outlines 
!pip install -q openai
!pip install -q datasets
!pip install -q trieregex
!pip install -q transformers==4.35.0
!pip install -q accelerate==0.24.1
!pip install -q bitsandbytes==0.41.0
!pip install -q torch==2.0.0

# !pip install -q bitsandbytes -U
# !pip install -q accelerate -U

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.0.3 which is incompatible.
tensorflow 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.15.0 requires huggingface-hub>=0.18.0, but you have huggingface-hub 0.17.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
outlines 0.0.14 requires torch>=2.1, but you have torch 2.0.0 which is incompatible.


## check

In [1]:
!pip freeze | grep accelerate

accelerate==0.24.1


In [2]:
!pip freeze | grep bitsandbytes

bitsandbytes==0.41.0


In [3]:
!pip freeze | grep transformers

transformers==4.35.0


In [4]:
!pip freeze | grep torch

torch==2.0.0
torchvision @ file:///home/conda/feedstock_root/build_artifacts/torchvision-split_1699863736598/work


In [8]:
# you need to restart the kernel after installing these libraries. especially accelerate

In [9]:
#g4.2x works but it's slow 
#g5.12x works and is faster

In [10]:
#mistral works and fits when 4bit . zeypher works faster

In [11]:
#g5.2xlarge runs mistral fast enough and it's roughly a dollar and a bit per hour

# Mistral

In [6]:
import torch
device = 'cuda'

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-v0.1", load_in_4bit=True, use_cache=False, torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

In [18]:
import torch
device = "cuda" # the device to load the model onto

prompt = \
"""
Example 
My favourite food is 
Complete the sentence with suitable word

My favourite food is potatoes
======
My favourite condiment is 

End 
"""

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
model#.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
out = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]
out

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> \nExample \nMy favourite food is \nComplete the sentence with suitable word\n\nMy favourite food is potatoes\n======\nMy favourite condiment is \n\nEnd \n==\nMy name is Anuj Garg\nMy name is anuj garg\ni am anuj garg\nmy name is anujgarg\n\n# this is the output in the console\n\n\n##this is the first string\n\n### This is another comment\n\n\n#This\n\n\n#This\n\n\n\nimport re\n\ndef freq(p1,p2):\n\tre4=re.findall(r"(.)'

## not quantised

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", torch_dtype=torch.bfloat16, device_map="auto")#, offload_folder="offload")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
model#.to(device)
# shoulnd't move to device when device map is auto 

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> My favourite condiment is mayo, the less fat content the better! I tend to always use Hellmans light version, because it never separates like many of the other less good quality brands here in the UK and I just love its flavour!\n\nWell, I have fallen in love with Hellmann’s light mayo, ever since I first was given a bottle as a child, and the more flavour it has the better! Now, because I know I cant get mayonnaise in the US, I'

# Zephy

In [9]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")



Loading checkpoint shards: 100%|██████████| 8/8 [00:49<00:00,  6.15s/it]


In [10]:
print('hello')

hello


In [12]:
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# Ah, me hearty matey! But yer question be a puzzler! A human cannot eat a helicopter in one sitting, as helicopters are not edible. They be made of metal, plastic, and other materials, not food!


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Ahoy there, matey! A human cannot eat a helicopter in one sitting, for a helicopter is not a food item. A helicopter is a mechanical device for flying. But if ye're talkin' 'bout chicken or beef, then I can tell ye that a human can consume a decent amount in one sitting, but not an entire helicopter. Yo ho ho!


In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch 

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
device = 'cuda'
prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
model#.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

'<s> My favourite condiment is hoisin sauce ! Not all sauce are the same though, as some are more sweeter, salty, tangier than others, or even thicker, with ingredients like pineapple or soy bean added. I’ve tried and made my way through some pretty terrible sauces, from cheap ones that I bought, as well as some really yummy sauces too, which I am more than happy to share. Here for your reading pleasures is a collection of Chinese sweet and sour'

# LLama

In [8]:
!pip install -q --upgrade huggingface_hub
!pip install -q python-dotenv
!pip install -q ipywidgets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [13]:
from huggingface_hub import login
import os
login(token=os.getenv('HUGGINGFACE_TOKEN'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [14]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-hf")

tokenizer_config.json: 100%|██████████| 776/776 [00:00<00:00, 5.44MB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 210MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 5.43MB/s]
config.json: 100%|██████████| 610/610 [00:00<00:00, 3.75MB/s]
model.safetensors.index.json: 100%|██████████| 33.4k/33.4k [00:00<00:00, 135MB/s]
model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]
model-00001-of-00003.safetensors:   0%|          | 21.0M/9.95G [00:00<00:47, 207MB/s]
model-00001-of-00003.safetensors:   0%|          | 41.9M/9.95G [00:00<00:49, 202MB/s]
model-00001-of-00003.safetensors:   1%|          | 73.4M/9.95G [00:00<00:42, 233MB/s]
model-00001-of-00003.safetensors:   1%|          | 105M/9.95G [00:00<00:41, 236MB/s] 
model-00001-of-00003.safetensors:   1%|▏         | 136M/9.95G [00:00<00:43, 227MB/s]
model-00001-of-00003.safetensors:   2%|▏         | 168M/9.95G [00:00<00:45, 216MB/s]
model-00001-of-00003.safetensors:   2%|▏         | 199M/

In [16]:
prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

OutOfMemoryError: CUDA out of memory. Tried to allocate 270.00 MiB (GPU 0; 22.20 GiB total capacity; 21.42 GiB already allocated; 73.12 MiB free; 21.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF